In [1]:
import sys

sys.path.append('/Users/leportella/src/serenata/medidor-de-poder/api/candidates')

from decouple import config
from datetime import datetime
import pandas as pd
import mongoengine as me

from models.deputy import DeputyInfo, Term
from models import Candidates

me.connect('serenata')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## Getting deputies complete information

In [2]:
tse_candidates = pd.read_csv('../data/2017-07-20-tse-candidates.xz', low_memory=False)
tse_candidates.head()

,year,phase,description,state,location,post,name,electoral_id,cpf,voter_id,result
0,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,JOÃO DE ALMEIDA,6,18139124249,000356682496,not_elected
1,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,PAULO CÉSAR FERREIRA DE ARAUJO,1,07857136204,000525772445,not_elected
2,2004,1,ELEICOES 2004,AC,ACRELANDIA,mayor,SEBASTIÃO BOCALOM RODRIGUES,27,17357152987,002053882488,elected
3,2004,1,ELEICOES 2004,AC,ACRELANDIA,vice_mayor,ERAIDES CAETANO DE SOUZA,7,40917860900,002817512496,NaN
4,2004,1,ELEICOES 2004,AC,ACRELANDIA,vice_mayor,RICARDO MODESTO DE OLIVEIRA,2,61109029268,002600402445,NaN


In [3]:
tse_candidates.result.value_counts()

alternate                 1108079
not_elected                322133
elected                    164835
elected_by_party_quota      92913
rejected                     9622
replaced                      694
runoff                        674
Name: result, dtype: int64

In [4]:
# select only candidates to federal deputy
deputy_candidates = tse_candidates[
    (tse_candidates.post=='federal_deputy') & 
    (tse_candidates.result.isin(['elected', 'elected_by_party_quota']))]

# select wanted features
#deputy_candidates = deputy_candidates[['name', 'state', 'cpf', 'voter_id', 'party']]

# rename column
deputy_candidates.rename(columns={'name': 'civil_name'}, inplace=True)

# remove duplicates
deputy_candidates.drop_duplicates(subset='cpf', inplace=True)
deputy_candidates.head(2)

/Users/leportella/.virtualenvs/serenata/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/leportella/.virtualenvs/serenata/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,year,phase,description,state,location,post,civil_name,electoral_id,cpf,voter_id,result
401825,2006,1,ELEICOES 2006,AC,ACRE,federal_deputy,FERNANDO MELO DA COSTA,10033,06567460253,000933682461,elected
401826,2006,1,ELEICOES 2006,AC,ACRE,federal_deputy,FLAVIANO FLÁVIO BAPTISTA DE MELO,10190,33251797700,000745162429,elected


In [5]:
deputy_candidates[deputy_candidates.civil_name=='EDSON MOREIRA DA SILVA']

,year,phase,description,state,location,post,civil_name,electoral_id,cpf,voter_id,result
1322586,2014,1,Eleições Gerais 2014,MG,MINAS GERAIS,federal_deputy,EDSON MOREIRA DA SILVA,130000000439,22063625115,109804380248,elected_by_party_quota


In [6]:
deputies = pd.read_csv('../data/2017-05-29-deputies.xz', dtype={'congressperson_id':  str, 
                                                                'congressperson_document': str,
                                                            })
#deputies = deputies[[
#    'congressperson_id', 
#    'congressperson_document', 
#    'civil_name', 
#    'congressperson_name',
#    'gender',
#    'state',
#    'party'
#]]
deputies.head(2)

,congressperson_id,budget_id,condition,congressperson_document,civil_name,congressperson_name,picture_url,gender,state,party,phone_number,email
0,178980,3283.0,Holder,361,ANTONIO GOULART DOS REIS,GOULART,http://www.camara.gov.br/internet/deputado/ban...,male,SP,PSD,3215-5533,dep.goulart@camara.gov.br
1,141335,2152.0,Holder,19,JOSÉ ROBERTO OLIVEIRA FARO,BETO FARO,http://www.camara.gov.br/internet/deputado/ban...,male,PA,PT,3215-5723,dep.betofaro@camara.gov.br


In [7]:
deputies_personal_info = pd.merge(deputy_candidates, deputies, how='inner', on=['civil_name', 'state'])
deputies_personal_info.head()

,year,phase,description,state,location,post,civil_name,electoral_id,cpf,voter_id,...,congressperson_id,budget_id,condition,congressperson_document,congressperson_name,picture_url,gender,party,phone_number,email
0,2006,1,ELEICOES 2006,AC,ACRE,federal_deputy,FLAVIANO FLÁVIO BAPTISTA DE MELO,10190,33251797700,000745162429,...,141434,2424.0,Holder,54,FLAVIANO MELO,http://www.camara.gov.br/internet/deputado/ban...,male,PMDB,3215-5224,dep.flavianomelo@camara.gov.br
1,2006,1,ELEICOES 2006,AP,AMAPÁ,federal_deputy,JANETE MARIA GÓES CAPIBERIBE,10210,18085830272,000069332577,...,73926,1135.0,Holder,10,JANETE CAPIBERIBE,http://www.camara.gov.br/internet/deputado/ban...,female,PSB,3215-5209,dep.janetecapiberibe@camara.gov.br
2,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,ALICE MAZZUCO PORTUGAL,10518,12377392504,024700970558,...,74057,1331.0,Holder,180,ALICE PORTUGAL,http://www.camara.gov.br/internet/deputado/ban...,female,PCdoB,3215-5420,dep.aliceportugal@camara.gov.br
3,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,DANIEL GOMES DE ALMEIDA,10520,07894090549,030538180515,...,74060,1339.0,Holder,188,DANIEL ALMEIDA,http://www.camara.gov.br/internet/deputado/ban...,male,PCdoB,3215-5317,dep.danielalmeida@camara.gov.br
4,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,JOSÉ ALVES ROCHA,10205,04789113515,047135190507,...,74554,3262.0,Holder,203,JOSÉ ROCHA,http://www.camara.gov.br/internet/deputado/ban...,male,PR,3215-5908,dep.joserocha@camara.gov.br


In [8]:
deputies_personal_info

,year,phase,description,state,location,post,civil_name,electoral_id,cpf,voter_id,...,congressperson_id,budget_id,condition,congressperson_document,congressperson_name,picture_url,gender,party,phone_number,email
0,2006,1,ELEICOES 2006,AC,ACRE,federal_deputy,FLAVIANO FLÁVIO BAPTISTA DE MELO,10190,33251797700,000745162429,...,141434,2424.0,Holder,54,FLAVIANO MELO,http://www.camara.gov.br/internet/deputado/ban...,male,PMDB,3215-5224,dep.flavianomelo@camara.gov.br
1,2006,1,ELEICOES 2006,AP,AMAPÁ,federal_deputy,JANETE MARIA GÓES CAPIBERIBE,10210,18085830272,000069332577,...,73926,1135.0,Holder,10,JANETE CAPIBERIBE,http://www.camara.gov.br/internet/deputado/ban...,female,PSB,3215-5209,dep.janetecapiberibe@camara.gov.br
2,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,ALICE MAZZUCO PORTUGAL,10518,12377392504,024700970558,...,74057,1331.0,Holder,180,ALICE PORTUGAL,http://www.camara.gov.br/internet/deputado/ban...,female,PCdoB,3215-5420,dep.aliceportugal@camara.gov.br
3,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,DANIEL GOMES DE ALMEIDA,10520,07894090549,030538180515,...,74060,1339.0,Holder,188,DANIEL ALMEIDA,http://www.camara.gov.br/internet/deputado/ban...,male,PCdoB,3215-5317,dep.danielalmeida@camara.gov.br
4,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,JOSÉ ALVES ROCHA,10205,04789113515,047135190507,...,74554,3262.0,Holder,203,JOSÉ ROCHA,http://www.camara.gov.br/internet/deputado/ban...,male,PR,3215-5908,dep.joserocha@camara.gov.br
5,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,JOSÉ CARLOS ALELUIA COSTA,10212,01782037500,021604240507,...,74553,1718.0,Holder,207,JOSÉ CARLOS ALELUIA,http://www.camara.gov.br/internet/deputado/ban...,male,DEM,3215-5854,dep.josecarlosaleluia@camara.gov.br
6,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,JOÃO CARLOS PAOLILO BACELAR FILHO,10230,15345931879,050482870540,...,141458,1270.0,Holder,187,JOÃO CARLOS BACELAR,http://www.camara.gov.br/internet/deputado/ban...,male,PR,3215-5928,dep.joaocarlosbacelar@camara.gov.br
7,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,JUTAHY MAGALHÃES JÚNIOR,10135,10660410591,012449470540,...,74570,3477.0,Holder,206,JUTAHY JUNIOR,http://www.camara.gov.br/internet/deputado/ban...,male,PSDB,3215-5407,dep.jutahyjunior@camara.gov.br
8,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,NELSON VICENTE PORTELA PELLEGRINO,10426,24289612504,037648360558,...,74571,3564.0,Holder,209,NELSON PELLEGRINO,http://www.camara.gov.br/internet/deputado/ban...,male,PT,3215-5826,dep.nelsonpellegrino@camara.gov.br
9,2006,1,ELEICOES 2006,BA,BAHIA,federal_deputy,ROBERTO PEREIRA DE BRITTO,10224,11257741500,029335590507,...,141529,2470.0,Holder,211,ROBERTO BRITTO,http://www.camara.gov.br/internet/deputado/ban...,male,PP,3215-5733,dep.robertobritto@camara.gov.br


In [9]:
deputies_personal_info.columns

Index(['year', 'phase', 'description', 'state', 'location', 'post',
       'civil_name', 'electoral_id', 'cpf', 'voter_id', 'result',
       'congressperson_id', 'budget_id', 'condition',
       'congressperson_document', 'congressperson_name', 'picture_url',
       'gender', 'party', 'phone_number', 'email'],
      dtype='object')

In [10]:
deputies_personal_info.to_csv('../data/datalake/deputy_info.csv', index=False)

# Adding info on MongoDB

In [11]:
def add_by_cpf(row):
    return Candidates(
        cpf=row['cpf'],
        civil_name=row['civil_name'],
        voter_id=row['voter_id'],
        state=row['state'],
    )

In [12]:
c = 0

deputies_personal_info.fillna(0, inplace=True)
for i, row in deputies_personal_info[c:].iterrows():

    try:
        candidate = Candidates.objects.get(cpf=row['cpf'])
        print(c, ' - Found by cpf')
    except:
        candidate = add_by_cpf(row)
        print(c, ' - Added by cpf: ', row['cpf'])  

    candidate.gender = row['gender']
    candidate.phone_number = row['phone_number']
    candidate.email = row['email']
    candidate.picture_url = row['picture_url']
    
    if not row['party'] in candidate.parties:
        candidate.parties.append(row['party'])
    
    deputy_info = DeputyInfo(
        congressperson_id=row['congressperson_id'],
        congressperson_name=row['congressperson_name'],
    )
    
    term = Term(begin=2015, end=2018, congressperson_document=row['congressperson_document'])
    deputy_info.terms.append(term)
    
    candidate.deputy_info = deputy_info
    candidate.save()
    c += 1

0  - Found by cpf
1  - Found by cpf
2  - Found by cpf
3  - Found by cpf
4  - Found by cpf
5  - Found by cpf
6  - Found by cpf
7  - Found by cpf
8  - Found by cpf
9  - Found by cpf
10  - Found by cpf
11  - Found by cpf
12  - Found by cpf
13  - Found by cpf
14  - Found by cpf
15  - Found by cpf
16  - Found by cpf
17  - Found by cpf
18  - Found by cpf
19  - Found by cpf
20  - Found by cpf
21  - Found by cpf
22  - Found by cpf
23  - Found by cpf
24  - Found by cpf
25  - Found by cpf
26  - Found by cpf
27  - Found by cpf
28  - Found by cpf
29  - Found by cpf
30  - Found by cpf
31  - Found by cpf
32  - Found by cpf
33  - Found by cpf
34  - Found by cpf
35  - Found by cpf
36  - Found by cpf
37  - Found by cpf
38  - Found by cpf
39  - Found by cpf
40  - Found by cpf
41  - Found by cpf
42  - Found by cpf
43  - Found by cpf
44  - Found by cpf
45  - Found by cpf
46  - Found by cpf
47  - Found by cpf
48  - Found by cpf
49  - Found by cpf
50  - Found by cpf
51  - Found by cpf
52  - Found by cpf
53 